In [1]:
import numpy as np
import scipy as sp
import scipy.linalg as spla
import scipy.sparse as sps
import math
from importlib import reload
from functools import reduce

#Load various data and functions
import fundamental_gates #Useful single qutrit gates
import interaction_and_decay #parameters - change dephasing rates in here
import embed_functions #functions to embed 1/2-qutrit objects into many-qutrit operators
import gate_sequences #gate sequences for various gates we've discussed - ZZ4 gate sequence is the one actually used in expt
import evolution #functions to calculate evolution of initial state under gate sequence
import teleport

import state_prep
import sim_tools

In [2]:
rho_EPR_07 = state_prep.rho_EPR_07_expt
rho_EPR_56 = state_prep.rho_EPR_56_expt
F_EPR_meas = 0.92
decoh = True
dephas = True

In [3]:
#Perform protocol for each state in the two-design states
Probabilities = []
Fidelities = []
print("ZZ4 Teleportation protocol fidelities: \n")
for ind, state in enumerate(state_prep.two_design_states):
    P_psi, F_psi = teleport.getPandF(gate_sequences.EF_scrambler_gatelist,
                            state,
                            rho_EPR_07,
                            rho_EPR_56,
                            F_EPR_meas,
                            decoh=decoh,
                            dephas=dephas)
    Probabilities.append(P_psi.real)
    Fidelities.append(F_psi.real)
    
teleport.print_report(Probabilities, Fidelities)

ZZ4 Teleportation protocol fidelities: 

P_psi = [0.123, 0.106, 0.104, 0.113, 0.112, 0.108, 0.109, 0.114, 0.111, 0.106, 0.109, 0.119]
F_psi = [0.707, 0.646, 0.622, 0.593, 0.591, 0.537, 0.524, 0.606, 0.554, 0.522, 0.56, 0.618]
<P> = 0.11109976742692462
<F> = 0.5901643399342041
<FP>/<P> = 0.5915573834586867




Perfect EPR prep, no decoherence or dephasing, perfect EPR measurement: 100%

Perfect EPR prep on Q5Q6, Imperfect EPR prep on Q0Q7, no decoherence or dephasing, perfect EPR measurement: 88%

Imperfect EPR prep on Q5Q6, Perfect EPR prep on Q0Q7, no decoherence or dephasing, perfect EPR measurement: 86%

Perfect EPR prep, no decoherence or dephasing, imperfect EPR measurement: 94%

Perfect EPR prep, with decoherence and dephasing, perfect EPR measurement: 81%

All sources of error: 62%

Multiply all single-error budgets: 57%

Everything bad except EPR measurement is perfect: 64%

Everything bad except EPR 07 preparation is perfect: 68%

Everything bad except EPR 56 preparation is perfect: 69%

Everything bad except no decoherence or dephasing: 73%

So it does seem like decoherence is our worst enemy right now, which sucks because that's the one thing we can't really change.  